In [1]:
from sklearn.datasets import load_digits

In [2]:
import numpy as np
from sklearn.externals import joblib

In [3]:
from sklearn.datasets.mldata import fetch_mldata
mnist = fetch_mldata('mnist-original')#, data_home='./data')

In [4]:
#mnist['data'][0]#.keys()

In [10]:
showpic(mnist['data'][550],n=28)

In [6]:

#X, y = load_digits(n_class=1,return_X_y=True)
X=mnist["data"]
y=mnist["target"]

In [7]:
from PIL import Image

In [23]:
def showdigit(index,multiplier=70,size=200, n = 28):
    tstimage1= Image.fromarray(X[index].reshape((n,n)) * multiplier)
    tstimage1 = tstimage1.convert('L')
    tstimage1 = tstimage1.resize((size,size))
    tstimage1.show()

In [24]:
def showpic(digit,multiplier=70,size=200, n=28):
    tstimage1= Image.fromarray(digit.reshape((n,n)) * multiplier)
    tstimage1 = tstimage1.convert('L')
    tstimage1 = tstimage1.resize((size,size))
    tstimage1.show()

In [11]:
from keras.models import Model, Sequential
from keras.layers import Flatten,Dropout, Dense, Input,merge

Using Theano backend.


In [36]:

input_dist = Input(shape=(784,),name = "input_dist")
input_data = Input(shape=(784,), name = "input_data")


encoder_1 = Dense(784,activation = 'relu', name = "encoder_1")
encoder_2 = Dense(300,activation = 'relu', name = "encoder_2")
encoder_3 = Dense(20,activation = 'softmax', name = "encoder_3")

encoder = encoder_1(input_data)
encoder = encoder_2(encoder)
encoder_out = encoder_3(encoder)

decoder_1 = Dense(200,activation = 'relu',name = "decoder_1")
decoder_2 = Dense(784,activation = 'relu', name = "decoder_2")

decoder = decoder_1(encoder_out)
decoder_out = decoder_2(decoder)

generator_1 = Dense(20,activation = 'relu', name = "generator_1")

generator_out = generator_1(input_dist)


merged_vector = merge([encoder_out,generator_out], mode='sum',output_shape=(20,), name = "merge")

#predictions1 = Dense()
predictions = Dense(1, activation='relu')(merged_vector)

model = Model(input=[input_data,input_dist], output= [predictions,decoder_out])
#just_autoencoder = Model(input=input_data, output= decoder_out)

In [13]:
#model=joblib.load("Sigmoid20epoch5gen.pkl")

In [15]:
for i in model.layers:
    print(i.name)#[4].trainable

input_data
encoder_1
encoder_2
input_dist
encoder_3
generator_1
decoder_1
merge
decoder_2
dense_1


In [16]:
def generator_error(y_true,y_pred):
    #y_pred = np.array(y_pred)
    return np.log(np.ones(y_pred.shape)-y_pred).sum()

In [17]:
from theano.tensor import basic as tensor, subtensor
def generator_error_theano(y_true,y_pred):
    '''
    TODO write normalisation
    '''
    return -tensor.log(1.0 - y_pred)  #-(target * tensor.log(output) + (1.0 - target) *
    #return -tensor.log(1.0 + (y_pred-1.0))

In [18]:
def tester_error(y_true,y_pred):
    y_pred = np.array(y_pred)
    #log(pred) if true = 0
    #log(1-pred) if true=1
    return np.log(y_true - y_pred*((-1)**(y_true+1))).sum()

In [19]:
def tester_error_theano(target,output):
    '''
    y_true = 0 -> it was real
    '''
    
    return -((target) * tensor.log(output) + (1.0 - target) * tensor.log(1.0 - output))



In [20]:
def generate_input(length=64,bottom_border =0 ,top_border=1):
    '''
    generates list of length length with trash from custom distribution.
    '''
    return 0

In [21]:
npx = np.array(X)

In [29]:
showpic(X[30])


In [27]:
bottom = np.array(X).min()
top = np.array(X).max()

In [28]:
X = X/top

In [30]:
top

255

In [313]:
generator_layers = 1
tester_train_steps = 1
n_epoch = 500
xnum=0

In [314]:
from keras.optimizers import SGD

opt = SGD(lr = 0.00001)

In [31]:
#just_autoencoder.compile("adam" , loss = "mse")

In [32]:
X.shape

(70000, 784)

In [33]:
#just_autoencoder.fit(X,X,nb_epoch = 2,verbose = 1)

In [343]:
#showpic(just_autoencoder.predict(X[50003].reshape((-1,784))),n=28)

In [283]:
#showpic(just_autoencoder.predict(X[11].reshape((-1,64))))

In [34]:
#showpic(just_autoencoder.predict(X[50023].reshape((-1,784))),n=28)

Updates: autoencoder -> guesser -> generator

In [35]:
for epoch in range(0,n_epoch):
    
    print("Epoch ",epoch)#,"Iter ",xnum)
    for xnum in range(0,len(X)):
        
        #freezing generator
        for i in model.layers:
            if "generator" in i.name:
                i.trainable = False
        #freezing tester
        model.layers[-1].trainable = False
        
        model.compile(opt , loss = [Nobody_Cares,"mse"])
        
        for i in range(num_encoder_train):
            #first data then dist
            model.train_on_batch([np.array(X[xnum]).reshape((-1,64)),np.zeros((64,)).reshape((-1,64))])
        
        #unfreezing tester
        model.layers[-1].trainable = True
        
        #freezing encoder
        for i in model.layers:
            if "encoder" or "decoder" in i.name:
                i.trainable = False
                
        model.compile(opt , loss = [tester_error_theano,Nobody_Cares])
        #doing tester_train_steps of tester training
        for k in range(0,tester_train_steps):
            #for xnum in range(0,len(X)):
            model.train_on_batch([np.array(X[xnum]).reshape((-1,64)),np.zeros((64,)).reshape((-1,64))],np.array([0.0])) #inputs original image
            model.train_on_batch([np.zeros((64,)).reshape((-1,64)),np.random.uniform(bottom,top,64).reshape((-1,64))],np.array([1.0])) #inputs noise to generator
            

        #Unfreezing generator        
        for i in model.layers:
            if "generator" in i.name:
                i.trainable = True
        #freezing tester
        model.layers[-1].trainable = False
        model.compile(opt, loss = [generator_error_theano,Nobody_Cares])
        #doing ? steps of generator training
        for nn in range(0,3):
            model.train_on_batch([np.zeros((64,)).reshape((-1,64)),np.random.uniform(bottom,top,64).reshape((-1,64))],np.array([1.0])) #inputs noise to generator

NameError: name 'n_epoch' is not defined

In [139]:
pred = model.predict([np.random.uniform(bottom,top,64).reshape((-1,64)),np.zeros((64,)).reshape((-1,64))])

In [146]:

pred2 = model.predict([np.zeros((64,)).reshape((-1,64)),np.array(X[44]).reshape((-1,64))])

In [147]:
pred2

array([[ 0.024002]], dtype=float32)

In [122]:
import theano
get_activations = theano.function([model.layers[0].input], model.layers[1].output, allow_input_downcast=True)
#activations = get_activations(X_batch) # same result as above

In [150]:
tst = np.random.uniform(bottom,top,64).reshape((-1,64))
showpic(tst,size=500,multiplier = 20)
Generated = get_activations(tst)
showpic(Generated,size=500,multiplier = 20)
showdigit(np.random.randint(low=0,high=120),size=500,multiplier = 20)

In [151]:
joblib.dump(model,"Sigmoid500epoch3gen.pkl",compress=9)

['Sigmoid500epoch3gen.pkl']